## Redes sociais - APS(1)
Fernando Fincatti, Gabriela Moreno e Beatriz Bernardino

---

### Import libraries
---

In [17]:
import pandas as pd
import csv
from graph_tool import Graph
import os
from graph_tool import util

### Auxiliary functions
---

In [18]:
def filter_by_year(path, year):
    """
        Receives the path for the dataset (csv) and the year that must be filter
        Return a list containing all the data to the desired year.
    """
    
    response = [] #list grouping all rows
    with open(path) as file:
        reader = csv.reader(file, delimiter=',', quotechar='"')
        i = 0;
        for row in reader:
            if i != 0: #jumps the header
                match_year = str(row[3]).split('/')[2] #get match year
                if match_year == year:
                    response.append(row)
            i += 1
        
    return response
            

### Analysing the dataset
---

In [19]:
df = pd.read_csv(r'./ATP Dataset_2012-01 to 2017-07_Int.csv')

In [20]:
df.head()

,ATP,Tournament,Tournament_Int,Date,Series,Series_Int,Court,Court_Int,Surface,Surface_Int,...,Winner,Winner_Int,Player1,Player1_Int,Player2,Player2_Int,Player1_Rank,Player2_Rank,Player1_Odds,Player2_Odds
0,1,BrisbaneInternational,1.2757,1/1/17,ATP250,2.9693,Outdoor,3.6494,Hard,4.4983,...,ThompsonJ.,6.7926,ThompsonJ.,6.7926,YmerE.,6.0375,79.0,160.0,1.29,3.50
1,1,BrisbaneInternational,1.2757,1/2/17,ATP250,2.9693,Outdoor,3.6494,Hard,4.4983,...,MahutN.,6.9297,MahutN.,6.9297,RobertS.,6.9152,39.0,54.0,1.54,2.43
2,1,BrisbaneInternational,1.2757,1/2/17,ATP250,2.9693,Outdoor,3.6494,Hard,4.4983,...,FerrerD.,6.3881,FerrerD.,6.3881,TomicB.,6.6345,21.0,26.0,2.01,1.77
3,1,BrisbaneInternational,1.2757,1/2/17,ATP250,2.9693,Outdoor,3.6494,Hard,4.4983,...,EdmundK.,6.8384,EdmundK.,6.8384,EscobedoE.,6.1645,45.0,141.0,1.37,3.01
4,1,BrisbaneInternational,1.2757,1/2/17,ATP250,2.9693,Outdoor,3.6494,Hard,4.4983,...,DimitrovG.,6.5157,DimitrovG.,6.5157,JohnsonS.,6.1087,17.0,33.0,1.41,2.85


In [21]:
df.describe()

,ATP,Tournament_Int,Series_Int,Court_Int,Surface_Int,Round_Int,Best_of,Winner_Int,Player1_Int,Player2_Int,Player1_Rank,Player2_Rank,Player1_Odds,Player2_Odds
count,14746.000000,14746.000000,14746.000000,14746.000000,14746.000000,14746.000000,14746.000000,14746.000000,14746.000000,14746.000000,14740.000000,14714.000000,14735.000000,14735.000000
mean,31.636240,1.482631,2.765566,3.583026,4.424294,5.239293,3.395361,6.516963,6.516963,6.520327,55.303189,88.492728,1.832519,3.705629
std,17.643054,0.285657,0.288073,0.146906,0.255291,0.180624,0.796527,0.287258,0.287258,0.295260,71.680632,127.036584,1.171306,3.417485
min,1.000000,1.007100,2.257400,3.257900,4.112900,5.125700,3.000000,6.009000,6.009000,6.000300,1.000000,1.000000,1.010000,1.010000
25%,19.000000,1.240800,2.648600,3.649400,4.112900,5.125700,3.000000,6.253700,6.253700,6.256000,15.000000,33.000000,1.220000,1.750000
50%,32.000000,1.432800,2.969300,3.649400,4.498300,5.135100,3.000000,6.496500,6.496500,6.509200,38.000000,62.000000,1.480000,2.600000
75%,47.000000,1.752900,2.969300,3.649400,4.498300,5.251100,3.000000,6.776300,6.776300,6.803300,72.000000,100.000000,2.040000,4.130000
max,66.000000,1.982000,2.970900,3.649400,4.919400,5.954200,5.000000,6.999700,6.999700,6.997200,1184.000000,7380.000000,23.450000,36.440000


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14746 entries, 0 to 14745
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ATP             14746 non-null  int64  
 1   Tournament      14746 non-null  object 
 2   Tournament_Int  14746 non-null  float64
 3   Date            14746 non-null  object 
 4   Series          14746 non-null  object 
 5   Series_Int      14746 non-null  float64
 6   Court           14746 non-null  object 
 7   Court_Int       14746 non-null  float64
 8   Surface         14746 non-null  object 
 9   Surface_Int     14746 non-null  float64
 10  Round           14746 non-null  object 
 11  Round_Int       14746 non-null  float64
 12  Best_of         14746 non-null  int64  
 13  Winner          14746 non-null  object 
 14  Winner_Int      14746 non-null  float64
 15  Player1         14746 non-null  object 
 16  Player1_Int     14746 non-null  float64
 17  Player2         14746 non-null 

### Creating fork
---

### a. o grafo é dirigido;
---

In [23]:
g = Graph(directed=True) # directed fork

### b. os vértices são jogadores que disputaram partidas em 2015
---

In [24]:
#filter data for 2015
data_2015 = filter_by_year('./ATP Dataset_2012-01 to 2017-07_Int.csv', '15')

In [25]:
jogadores = []
for listas in data_2015:
    jogadores.append(listas[15])
    jogadores.append(listas[17])
len(jogadores)

5260

In [26]:
jogadores = list(dict.fromkeys(jogadores))
len(jogadores)
jogadores

['DuckworthJ.',
 'SimonG.',
 'KokkinakisT.',
 'BenneteauJ.',
 'ChardyJ.',
 'GolubevA.',
 'TomicB.',
 'QuerreyS.',
 'KukushkinM.',
 'CopilM.',
 'KubotL.',
 'AndersonK.',
 'JohnsonS.',
 'MatosevicM.',
 'NieminenJ.',
 'KudlaD.',
 'KlizanM.',
 'MelzerJ.',
 'MillmanJ.',
 'WilliamsR.',
 'DolgopolovO.',
 'BerlocqC.',
 'GrothS.',
 'HewittL.',
 'NishikoriK.',
 'DimitrovG.',
 'RaonicM.',
 'FedererR.',
 'MullerG.',
 'RogerVasselinE.',
 'HaiderMaurerA.',
 'GranollersM.',
 'CoricB.',
 'HaaseR.',
 'YmerE.',
 'SijslingI.',
 'BerankisR.',
 'VanniL.',
 'VeselyJ.',
 'PrashanthV.',
 'BedeneA.',
 'LackoL.',
 'ItoT.',
 'RamanathanR.',
 'GarciaLopezG.',
 'DonskoyE.',
 'CarrenoBustaP.',
 'GonzalezA.',
 'LuY.H.',
 'DevvarmanS.',
 'GojowczykP.',
 'FallaA.',
 'GoffinD.',
 'WawrinkaS.',
 'BautistaR.',
 'LopezF.',
 'BolelliS.',
 'BeckerB.',
 'DodigI.',
 'SafwatM.',
 'GasquetR.',
 'AndujarP.',
 'BrownD.',
 'LorenziP.',
 'VerdascoF.',
 'GabashviliT.',
 'KarlovicI.',
 'RosolL.',
 'BerdychT.',
 'IstominD.',
 'SeppiA.

In [27]:
for _ in jogadores:
    g.add_vertex()

print('número de vértices de g:', g.num_vertices())

número de vértices de g: 284


### c. Cada vértice deve ter um atributo que representa o nome do jogador;
---

In [28]:
g.vp['nome'] = g.new_vp('object')

In [33]:
for indice in range(len(jogadores)):
    v = g.vertex(indice)
    g.vp['nome'][v] = jogadores[indice]
    ##print(g.vp['nome'][v])

### d. Uma aresta (n,m) existe se n venceu m uma ou mais vezes em 2015;
---

In [30]:
lista_vencedor_perdedor=[]
for valores in data_2015:
    lista=[]
    vencedor = valores[13]
    lista.append(vencedor)
    if vencedor == valores[15]:
        lista.append(valores[17])
    else:
        lista.append(valores[15])
    if lista not in lista_vencedor_perdedor
        lista_vencedor_perdedor.append(lista)
    else:
        
        
       
len(lista_vencedor_perdedor)

2312

In [31]:
for dupla in lista_vencedor_perdedor:
    vencedor = dupla[0]
    perdedor = dupla[1]
    str_index_vencedor = util.find_vertex(g,g.vp['nome'],vencedor) 
    index_vencedor = int(str(str_index_vencedor).split(" ")[4].replace("'",""))
    str_index_perdedor = util.find_vertex(g,g.vp['nome'],perdedor) 
    index_perdedor = int(str(str_index_perdedor).split(" ")[4].replace("'",""))
    g.add_edge(index_vencedor, index_perdedor)
print('número de arestas de g:', g.num_edges())

número de arestas de g: 2312


In [ ]:
g.ep['vitorias'] = g.new_ep('object')


In [32]:
from netpixi.data.gt import *
save_gt(g, 'exemplo.net.gz')
import netpixi
r = netpixi.render('exemplo.net.gz')